In [1]:
import os
import datetime

import numpy as np
import pandas as pd
import netCDF4

import bokeh
import bokeh.embed
import bokeh.plotting
import bokeh.io

In [2]:
bokeh.io.output_notebook()

Loading BokehJS ...

In [3]:
DATA_DIR = os.path.abspath(os.path.join(
    os.path.abspath(''), "..", "data"
))

In [4]:
print("reading data")
# [-100000:] subset for performance
s = slice(None, None)
ds = netCDF4.Dataset(os.path.join(DATA_DIR, "id1-DELFZL.nc"))
sea_surface_height = ds.variables["sea_surface_height"][0][s]
t_num = ds.variables["time"][s]
print("converting times to objects ")
t = netCDF4.num2date(t_num, ds.variables["time"].units)
print("closing dataset")
ds.close()

print("converting objects")

reading data
converting times to objects 
closing dataset
converting objects


In [5]:
# convert to pandas
df = pd.DataFrame(
    data={
        "sea_surface_height": sea_surface_height,
        "time": t
    }
)
df = df.set_index("time")

print("computing means")
df_days = df.resample("1D").agg([np.mean, np.amin, np.amax])
df_months = df.resample("1M").agg([np.mean, np.amin, np.amax])
df_years = df.resample("12M").agg([np.mean, np.amin, np.amax])

for df_i in [df_days, df_months, df_years]:
    df_i.columns = df_i.columns.droplevel(0)
    df_i["sea_surface_height"] = df_i["mean"]

for df_i in [df_days, df_months, df_years, df]:
    df_i["left"] = df_i.index
    df_i["right"] = df_i.index
    df_i.loc[0:-1, "right"] = df_i.index[1:]

df['mean'] = df['sea_surface_height']
df['amin'] = df['sea_surface_height']
df['amax'] = df['sea_surface_height']

computing means


In [6]:
print("generating sources")
source = bokeh.models.ColumnDataSource(data=df_days)
source_raw = bokeh.models.ColumnDataSource(data=df)
source_days = bokeh.models.ColumnDataSource(data=df_days)
source_months = bokeh.models.ColumnDataSource(data=df_months)
source_years = bokeh.models.ColumnDataSource(data=df_years)

print("generating plots")
p = bokeh.plotting.Figure(x_axis_type="datetime", title="Title")
for key in ["sea_surface_height", "mean"]:
    p.line("time", key, source=source)
p.quad(left="left", right="right", bottom="amin", top="amax", source=source, alpha=0.2)
p.xaxis.axis_label = 'time'
p.yaxis.axis_label = 'sea_surface_height'
p.x_range.name = 'time'
p.y_range.name = 'sea_surface_height'


generating sources
generating plots


In [7]:
bokeh.io.show(p)

In [8]:
bokeh.io.output_html("Delfzijl_test.html")
bokeh.io.show(p)

AttributeError: module 'bokeh.io' has no attribute 'output_html'